In [1]:
import numpy as np
import torch

# e.g.1

In [2]:
torch.ones(3), torch.zeros(3)[0]

(tensor([1., 1., 1.]), tensor(0.))

# e.g.2

In [3]:
triangle_points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
triangle_points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [4]:
triangle_points.shape

torch.Size([3, 2])

In [5]:
triangle_points[1]

tensor([5., 3.])

In [6]:
triangle_points[1][0]

tensor(5.)

In [7]:
triangle_points[None]

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

# e.g.3 - разделяет пробелами при выводе лучше чем numpy!

In [8]:
torch.zeros(3,2,4)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [9]:
torch.ones(3,2,4,5)

tensor([[[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]])

# семантически именованные измерения

In [10]:
img_t = torch.randn(3, 5, 5)                     # форма [каналы (цветовые), строки, столбцы]
batch_t = torch.randn(100, 3, 5, 5)              # форма [батч, каналы, строки, столбцы]
weights = torch.tensor([0.2126, 0.7152, 0.0722]) # форма [вероятности классов]

In [11]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

/tmp/ipykernel_17505/2371314847.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /pytorch/c10/core/TensorImpl.h:1939.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

## обычно семантический порядок одинаков с конца

In [12]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')

print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([100, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


### но одномерные измерения опускаются, имена позволяют их восстановить по семантике

In [13]:
weights_aligned = weights_named.align_as(img_named)

print('weights aligned:', weights_aligned.shape, weights_aligned.names)

weights aligned: torch.Size([3, 1, 1]) ('channels', 'rows', 'columns')


### теперь могу замешать цветовые каналы в оттенки серого

In [14]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [15]:
gray_naive = img_named.mean('channels')
gray_naive.shape, gray_naive.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [16]:
gray_naive = img_named.mean(-3)
gray_naive.shape, gray_naive.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [17]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

# PyTorch позволяет перемножать объекты одинаковой формы: 
## либо когда один из них имеет размер 1 по заданному измерению (тогда по этому измерению идет как умножение на скаляр)
## либо PyTorch автоматически добавляет (делает broadcasting на недостающие измерения) в начало списка индексов новые измерения размером 1 (тогда по ним тоже как умножение на скаляр)

In [18]:
mytest = torch.tensor([
    [
        [1,2,3],
        [4,5,6],
        [7,8,9],
        [10,11,12]
    ],
    [
        [13,14,15],
        [16,17,18],
        [19,20,21],
        [22,23,24]
    ]
])
mytest.shape

torch.Size([2, 4, 3])

In [19]:
weights_2 = torch.tensor([1,2])
weights_3 = torch.tensor([1,2,3])
weights_4 = torch.tensor([1,2,3,4])

### для таких пройдет только свертка по последнему измерению - при совпадении размерности по нему

In [20]:
try:
    mytest * weights_2
except BaseException as err_message:
    print('\nERROR FYI:', err_message,'\n')


ERROR FYI: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 2 



In [21]:
print('\nЭто пройдет равносильно приписыванию всех измерений слева с размерностью 1:')
mytest * weights_3


Это пройдет равносильно приписыванию всех измерений слева с размерностью 1:


tensor([[[ 1,  4,  9],
         [ 4, 10, 18],
         [ 7, 16, 27],
         [10, 22, 36]],

        [[13, 28, 45],
         [16, 34, 54],
         [19, 40, 63],
         [22, 46, 72]]])

In [22]:
weights_3.shape, weights_3.unsqueeze(0).unsqueeze(0).shape

(torch.Size([3]), torch.Size([1, 1, 3]))

In [23]:
mytest * (weights_3.unsqueeze(0).unsqueeze(0)) 

tensor([[[ 1,  4,  9],
         [ 4, 10, 18],
         [ 7, 16, 27],
         [10, 22, 36]],

        [[13, 28, 45],
         [16, 34, 54],
         [19, 40, 63],
         [22, 46, 72]]])

In [24]:
try:
    mytest * weights_4
except BaseException as err_message:
    print('\nERROR FYI:', err_message,'\n')


ERROR FYI: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 2 



### чтобы свернуть, надо сдвинуть свертываемое измерение на нужную позицию в списке индексов (либо использовать именование индексов и свертывать по имени индекса)

#### пример 1

In [25]:
weights_4.shape, weights_4.unsqueeze(-1).shape, weights_4.unsqueeze(-1).unsqueeze(0).shape

(torch.Size([4]), torch.Size([4, 1]), torch.Size([1, 4, 1]))

In [26]:
(mytest * (weights_4.unsqueeze(-1))).shape, (mytest * (weights_4.unsqueeze(-1).unsqueeze(0))).shape

(torch.Size([2, 4, 3]), torch.Size([2, 4, 3]))

In [27]:
(mytest * (weights_4.unsqueeze(-1)))

tensor([[[ 1,  2,  3],
         [ 8, 10, 12],
         [21, 24, 27],
         [40, 44, 48]],

        [[13, 14, 15],
         [32, 34, 36],
         [57, 60, 63],
         [88, 92, 96]]])

#### пример 2

In [28]:
weights_2.shape, weights_2.unsqueeze(-1).unsqueeze(-1).shape

(torch.Size([2]), torch.Size([2, 1, 1]))

In [29]:
(mytest * (weights_2.unsqueeze(-1).unsqueeze(-1))).shape

torch.Size([2, 4, 3])

In [30]:
mytest * (weights_2.unsqueeze(-1).unsqueeze(-1))

tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9],
         [10, 11, 12]],

        [[26, 28, 30],
         [32, 34, 36],
         [38, 40, 42],
         [44, 46, 48]]])

#### итак, справа измерения добавляем руками - так как это меняет их семантику, а слева добавляются автоматически - копируя семантику из тензора с большим кол-вом измерений (если именовать измерения, то и справа сможет добавиться автоматом - так как семантика обозначена в имени!)

# в numpy есть функция `einsum()` свертки по промежуточным индексам со строковым описанием позиций индексов, которая также перенесена в PyTorch https://rockt.github.io/2018/04/30/einsum:

In [31]:
weights.shape, img_t.shape, batch_t.shape

(torch.Size([3]), torch.Size([3, 5, 5]), torch.Size([100, 3, 5, 5]))

In [32]:
img_gray_weighted_fancy = torch.einsum('chw,c->hw', img_t, weights)           # свертка по индексу с
batch_gray_weighted_fancy = torch.einsum('bchw,c->bhw', batch_t, weights)     # свертка по индексу с

img_gray_weighted_fancy.shape, batch_gray_weighted_fancy.shape

(torch.Size([5, 5]), torch.Size([100, 5, 5]))

## `...` означает несколько индексов - так где это однозначно восстановимо:

In [33]:
img_named.shape, img_named[..., :3].shape, weights_named.shape

(torch.Size([3, 5, 5]), torch.Size([3, 5, 3]), torch.Size([3]))

In [34]:
img_named.names, img_named[..., :3].names, weights_named.names

(('channels', 'rows', 'columns'),
 ('channels', 'rows', 'columns'),
 ('channels',))

### поэтому если мы не уверены, сколько еще слева добавится измерений, можно их не специфицировать

In [35]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)         # свертка по индуксу в позиции с (третий справа)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)     # свертка по индуксу в позиции с (третий справа)

img_gray_weighted_fancy.shape, batch_gray_weighted_fancy.shape

(torch.Size([5, 5]), torch.Size([100, 5, 5]))

**получили то же самое, как при полной спецификации списка индексов выше**

# Эйнштейновская свертка работает как в физике:

$T_{ij}$

In [36]:
T_ij = np.array([
    [1,2],
    [3,4]
])
T_ij

array([[1, 2],
       [3, 4]])

$T^T = T_{ji}$

In [37]:
np.einsum('ij->ji',T_ij)

array([[1, 3],
       [2, 4]])

$\sum_j T_{ij}$

In [38]:
np.einsum('ij->i',T_ij)

array([3, 7])

$Trace(T)$

In [39]:
np.einsum('ii->i',T_ij), np.einsum('ii->i',T_ij).sum()

(array([1, 4]), np.int64(5))

# ХРАНИЛИЩЕ vs ИНДЕКСАЦИЯ

In [40]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

/tmp/ipykernel_17505/2923139482.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

## попробуем совет из ворнинга

In [41]:
points.untyped_storage()

 0
 0
 128
 64
 0
 0
 128
 63
 0
 0
 160
 64
 0
 0
 64
 64
 0
 0
 0
 64
 0
 0
 128
 63
[torch.storage.UntypedStorage(device=cpu) of size 24]

## г.м., может в бинарном будет виден смысл

In [42]:
count = 0
for B in points.untyped_storage():
    count += 1
    print(bin(B).split('b')[1].rjust(8,'0'))
    if count % 4 == 0:
        print('-'*10)

00000000
00000000
10000000
01000000
----------
00000000
00000000
10000000
00111111
----------
00000000
00000000
10100000
01000000
----------
00000000
00000000
01000000
01000000
----------
00000000
00000000
00000000
01000000
----------
00000000
00000000
10000000
00111111
----------


**хз все равно! Видно только, что второй и последний элементы совпадают**

## любуемся на офсеты

In [43]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [44]:
torch.tensor([[el for el in raw] for raw in points])

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [45]:
torch.tensor([[el.storage_offset() for el in raw] for raw in points])

tensor([[0, 1],
        [2, 3],
        [4, 5]])

## любуемся на размеры (совпадает с `shape`)

In [46]:
[[el.size() for el in raw] for raw in points]

[[torch.Size([]), torch.Size([])],
 [torch.Size([]), torch.Size([])],
 [torch.Size([]), torch.Size([])]]

In [47]:
[raw.size() for raw in points]

[torch.Size([2]), torch.Size([2]), torch.Size([2])]

In [48]:
points.size()

torch.Size([3, 2])

## любуемся на шаги (многомерные)

In [49]:
points.stride()

(2, 1)

# копирование при срезах

In [50]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [51]:
points_row = points[1]
points_row[0] = 10
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [52]:
points_row = points[1].clone()    # смена хранилища
points_row[0] = 5
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

# транспонирование и непрерывность тензора

In [53]:
points.t()

tensor([[ 4., 10.,  2.],
        [ 1.,  3.,  1.]])

In [54]:
points.t().stride()

(1, 2)

In [55]:
points_t = points.t().clone()
points_t

tensor([[ 4., 10.,  2.],
        [ 1.,  3.,  1.]])

In [56]:
points_t.stride()

(1, 2)

In [57]:
points.is_contiguous()

True

In [58]:
points.t().is_contiguous()

False

In [59]:
points_t.is_contiguous()

False

## как же вернуть непрерывность индексации транспонированному тензору (некоторые тензорные операции того требуют!)

In [60]:
points_t = points.t().contiguous()
points_t.is_contiguous()

True

In [61]:
points_t

tensor([[ 4., 10.,  2.],
        [ 1.,  3.,  1.]])

**внешне все одинаково**

In [62]:
points.stride() , points_t.stride()

((2, 1), (3, 1))

# использование GPU

In [63]:
points.device

device(type='cpu')

In [64]:
try:
    points_gpu = points.to(device='cuda')
except BaseException as err_message:
    print('\nERROR FYI:', err_message,'\n')

In [65]:
points.device

device(type='cpu')

In [66]:
points_gpu

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]], device='cuda:0')

# смена типов между pyTorch и Numpy - происходит при общем хранилище!:

In [67]:
points.numpy()

array([[ 4.,  1.],
       [10.,  3.],
       [ 2.,  1.]], dtype=float32)

**в типе видим, что размер флотов остался торчевым**

In [68]:
points_np = np.array([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_np

array([[4., 1.],
       [5., 3.],
       [2., 1.]])

In [69]:
points = torch.from_numpy(points_np)
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]], dtype=torch.float64)

**NB!: а вот тут прилетел дефолтный размер флотов из numpy**

In [70]:
points_np[1,0] = 10
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]], dtype=torch.float64)

**полезно, когда работаем с тензорами торч, но потребовалась непортированная функция нампай - можно не копировать весь тензор в объект numpy, а обратиться к нему по интерфейсу numpy и вызвать редкую функцию, а потом вернуться в объекту тензора pyTorch - в данных все будет ОК!**

# в заключение сериализация - сохранение и загрузка файла

In [71]:
torch.save(points, 'points.t')
torch.load('points.t', weights_only=True)

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]], dtype=torch.float64)

## вывод в формате HDF5

In [72]:
import h5py

**позволяет читать таблицы в память построчно и другие фишки HDF5**